# 2.0 Data Cleaning

In [1]:
# libraries
import os
import sys
import inspect
sys.path.append("../src")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from shared import directories
from shared import filenames
from shared import variables

sys.path.append(directories.ANALYSIS_DIR)
import description
pd.set_option('display.max_rows', 500)


## 2.1 Obtain raw data

In [2]:
def get_data():
    df = pd.read_csv("../data/raw/Speed Dating Data.csv",
                     encoding="Latin-1", low_memory=False)
    df_columns = pd.read_csv("../data/external/columns.csv",
                     encoding="Latin-1", low_memory=False)
    df_universities = pd.read_csv("../data/external/universities.csv",
                     encoding="Latin-1", low_memory=False)
                             
    return(df, df_columns, df_universities)
df, df_columns, df_universities = get_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB


## 2.2 Categorical Data

### 2.2.0 Audit

In [3]:
df_cat = df.select_dtypes(include=['object'])
description.describe_qual_df(df_cat)

,count,unique,top,freq,missing
field,8315,259,Business,521,63
undergra,4914,241,UC Berkeley,107,3464
mn_sat,3133,68,"1,400.00",403,5245
tuition,3583,115,"26,908.00",241,4795
from,8299,269,New York,522,79
zipcode,7314,409,0,355,1064
income,4279,261,"55,080.00",124,4099
career,8289,367,Finance,202,89


### 2.2.1 Categorical Data Processing Plan
The following variables must be converted to numeric:
1. mn_sat 
2. tuition
3. income 

In terms of missing data:
1. Field and career are 'form filled' and are subject to misspellings and omission. The field_cd and career_c fields will
be used instead.
2. There are no other opportunities for data imputation for the categorical variables.

#### 2.2.1.1 Convert categorical numbers to numeric

In [4]:
df['mn_sat'] = df['mn_sat'].str.replace(",","").astype(float)
df['income'] = df['income'].str.replace(",","").astype(float)
df['tuition'] = df['tuition'].str.replace(",","").astype(float)

## 2.3 Quantitative Data

### 2.3.0 Quantitative Data Audit
Looking preference and rating variables that require normalization due to different scoring instructions by wave 

### 2.3.1 Quantitative Data Processing Plan
1. id: Impute missing id.
2. gender: Recode as Categorical Variables
3. race: Recode as Categorical Variables
4. pid: Impute 10 missing pid values.
5. Change partner to pid and pid to piid for clarity.
6. field_cd: Update field_cd for 'Operations Research' to 8 for Business/Econ/Finance
7. normalize all ratings and preferences to 100 point scale.

Encoding: Use descriptive encoding for target variables
1. decision
2. decision_o
3. match

Encoding cross-references
Create label cross reference tables so that they are available for plotting.

In [5]:
df_columns = pd.read_csv("../data/external/columns.csv",
                 encoding="Latin-1", low_memory=False)

#### 2.3.1.1 Missing id
Since each subject has a unique iid, we can impute the missing id by finding the associated id for the same iid in another observation.

In [6]:
iid = df.loc[df['id'].isna()]['iid'].tolist()
id = df.loc[df['iid'] == iid]['id'].unique()
df.loc[(df.id.isna()), 'id'] = id[0]

#### 2.3.2.2  Recode Gender 
Recode gender to "Male" and "Female" for reporting

In [7]:
df['gender'] = np.where(df['gender'] == 0, "Female", "Male")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(177), int64(12), object(6)
memory usage: 12.5+ MB


#### 2.3.2.3 Race Encode

In [8]:
df['race'] = np.where(df['race'] == 1, 'Black',
                     np.where(df['race'] == 2, 'Caucasian',
                             np.where(df['race'] == 3, "Latino",
                                     np.where(df['race'] == 4, "Asian",
                                             np.where(df['race'] == 5, "Native American", "Other")))))

In [9]:
df['race_o'] = np.where(df['race_o'] == 1, 'Black',
                     np.where(df['race_o'] == 2, 'Caucasian',
                             np.where(df['race_o'] == 3, "Latino",
                                     np.where(df['race_o'] == 4, "Asian",
                                             np.where(df['race_o'] == 5, "Native American", "Other")))))

#### 2.3.1.2 Missing piid and Column Rename
First, rename partner to pid and pid to piid for clarity. Now we have 10 missing piid's - the partner's iid number. As it turns out, all 10 are for pid number 7, from wave 5. We can obtain the missing piid from the iid for id number 7 of the same wave. 

In [10]:
df = df.rename(columns={'pid': 'piid'})
df = df.rename(columns={'partner':'pid'})
wave_pid = df.loc[df['piid'].isna()][['wave','pid']].drop_duplicates()
piid = df.loc[(df['wave'] == wave_pid.wave.tolist()) & (df['id'] == wave_pid.pid.tolist())]['iid'].drop_duplicates().tolist()
df.loc[(df.piid.isna()), 'piid'] = piid

#### 2.3.2.3 Update Field_Cd for Operations Research

In [11]:
df.loc[df['field'] == 'Operations Research', 'field_cd'] = 8

#### 2.3.2.4 Career_c Missing Data

In [12]:
df.loc[df['career'] == 'lawyer', 'career_c'] = 1
df.loc[df['career'] == 'law', 'career_c'] = 1
df.loc[df['career'] == 'Economist', 'career_c'] = 7
df.loc[df['career'] == 'tech professional', 'career_c'] = 15
df.loc[df['career'].isnull(), 'career_c'] = 10 # NaNs converted to undecided

#### 2.3.2.5 Convert select binary variables to categorical

In [13]:
df['b_dec'] = df['dec']
df['b_dec_o'] = df['dec_o']
df['b_match'] = df['match']
df['dec'] = np.where(df['dec']==0, 'No', 'Yes')
df['dec_o'] = np.where(df['dec_o']==0, 'No', 'Yes')
df['match'] = np.where(df['match']==0, 'Not Matched', 'Matched')
df['condtn'] = np.where(df['condtn']==1, 'Limited Choice', 'Extensive Choice')
df['samerace'] = np.where(df['samerace']==1, 'Same Race', 'Not Same Race')
df['met'] = np.where(df['met']==1, 'Met', 'Not Met')
df['met_o'] = np.where(df['met_o']==1, 'Met', 'Not Met')

#### 2.3.2.6 Difference in Age [Male-Female]

In [14]:
df['age_diff'] = df['age']-df['age_o']

#### 2.3.2.7 Encoding Cross-Reference Table

In [15]:
# Field_cd
code = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,]
value = ['Law','Math','Social Science, Psychologist','Medical Science, Pharmaceuticals, and Bio Tech','Engineering',
         'English/Creative Writing/ Journalism','History/Religion/Philosophy','Business/Econ/Finance',
         'Education, Academia','Biological Sciences/Chemistry/Physics','Social Work','Undergrad/undecided',
         'Political Science/International Affairs','Film','Fine Arts/Arts Administration','Languages',
         'Architecture','Other']
df_labels = pd.DataFrame({'field_cd': code, 'field_label':value})
df = df.merge(df_labels, how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8378 entries, 0 to 8377
Columns: 200 entries, iid to field_label
dtypes: float64(174), int64(10), object(16)
memory usage: 12.8+ MB


In [16]:
# Goal
code = [1,2,3,4,5,6]
value = ['Seemed like a fun night out', 'To meet new people', 'To get a date', 'Looking for a serious relationship',
         'To say I did it', 'Other']
df_labels = pd.DataFrame({'goal': code, 'goal_label':value})
df = df.merge(df_labels, how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8378 entries, 0 to 8377
Columns: 201 entries, iid to goal_label
dtypes: float64(174), int64(10), object(17)
memory usage: 12.9+ MB


In [17]:
# Date
code = [1,2,3,4,5,6,7]
value = ['Several times a week','Twice a week','Once a week','Twice a month','Once a month','Several times a year',
         'Almost never']
df_labels = pd.DataFrame({'date': code, 'date_label':value})
df = df.merge(df_labels, how='left')

In [18]:
# Go_out
code = [1,2,3,4,5,6,7]
value = ['Several times a week','Twice a week','Once a week','Twice a month','Once a month','Several times a year',
         'Almost never']
df_labels = pd.DataFrame({'go_out': code, 'go_out_label':value})
df = df.merge(df_labels, how='left')

In [19]:
# Career_c
code = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
value = ['Lawyer','Academic/Research','Psychologist','Doctor/Medicine','Engineer','Creative Arts/Entertainment',
         'Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin','Real Estate',
         'International/Humanitarian Affairs','Undecided','Social Work','Speech Pathology','Politics',
         'Pro sports/Athletics','Other','Journalism','Architecture']
df_labels = pd.DataFrame({'career_c': code, 'career_label':value})
df = df.merge(df_labels, how='left')

In [20]:
# length
code = [1,2,3]
value = ['Too Little', 'Too Much', 'Just Right']
df_labels = pd.DataFrame({'length': code, 'length_label':value})
df = df.merge(df_labels, how='left')

In [21]:
# Numdat_2
code = [1,2,3]
value = ['Too Few', 'Too Many', 'Just Right']
df_labels = pd.DataFrame({'numdat_2': code, 'numdat_2_label':value})
df = df.merge(df_labels, how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8378 entries, 0 to 8377
Columns: 206 entries, iid to numdat_2_label
dtypes: float64(174), int64(10), object(22)
memory usage: 13.2+ MB


#### 2.3.2.8 Add Partner  Field and Same Field Variable

In [22]:
df_fields = df[['iid', 'field_cd']]
df_fields = df_fields.rename(columns={'iid': 'piid', 'field_cd':'field_cd_o'})
df_fields = df_fields.drop_duplicates()
df = pd.merge(df,df_fields, on='piid', how='left')
df['same_field'] = np.where(df['field_cd']==df['field_cd_o'], 'Yes', 'No')

#### 2.3.2.9 Add Partner Career and Same Career Variable

In [23]:
df_career = df[['iid', 'career_c']]
df_career = df_career.rename(columns={'iid': 'piid', 'career_c':'career_c_o'})
df_career = df_career.drop_duplicates()
df = pd.merge(df,df_career, on='piid', how='left')
df['same_career'] = np.where(df['career_c']==df['career_c_o'], 'Yes', 'No')

#### 2.3.2.9 Add Partner Career Label

In [24]:
code = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
value = ['Lawyer','Academic/Research','Psychologist','Doctor/Medicine','Engineer','Creative Arts/Entertainment',
         'Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin','Real Estate',
         'International/Humanitarian Affairs','Undecided','Social Work','Speech Pathology','Politics',
         'Pro sports/Athletics','Other','Journalism','Architecture']
df_labels = pd.DataFrame({'career_c_o': code, 'career_label_o':value})
df = df.merge(df_labels, how='left')

#### 2.3.2.10 Add Partner Income

In [25]:
df_income = df[['iid', 'income']]
df_income = df_income.rename(columns={'iid': 'piid', 'income':'income_o'})
df_income = df_income.drop_duplicates()
df = pd.merge(df,df_income, on='piid', how='left')
df['income_diff'] = df['income'] - df['income_o']

#### 2.3.2.11 Add Median SAT

In [26]:
df_sat = df[['iid', 'mn_sat']]
df_sat = df_sat.rename(columns={'iid': 'piid', 'mn_sat':'ms_sat_o'})
df_sat = df_sat.drop_duplicates()
df = pd.merge(df,df_sat, on='piid', how='left')

#### 2.3.2.12 Add Partner Tuition

In [27]:
df_tuition = df[['iid', 'tuition']]
df_tuition = df_tuition.rename(columns={'iid': 'piid', 'tuition':'tuition_o'})
df_tuition = df_tuition.drop_duplicates()
df = pd.merge(df,df_tuition, on='piid', how='left')

#### 2.3.2.13 Standardize University

In [28]:
df = pd.merge(df,df_universities, on='undergra', how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8378 entries, 0 to 8377
Columns: 216 entries, iid to undergra_cd
dtypes: float64(180), int64(10), object(26)
memory usage: 13.9+ MB


#### 2.3.2.14 Standardize SAT Scores by University

In [29]:
df.loc[df.undergra_cd == 'Wesleyan University','mn_sat'] = 1360

#### 2.3.2.15 Add University Rank 

In [30]:
mn_sat = df['mn_sat'].drop_duplicates()
mn_sat = mn_sat.dropna()
mn_sat = pd.DataFrame({'mn_sat':sorted(mn_sat, reverse=True)})
mn_sat['uni_rank'] = mn_sat.rank(ascending=False)
mn_sat['top_pct'] = mn_sat['uni_rank'] / max(mn_sat['uni_rank']) * 100
mn_sat['uni_rank'] = np.where(mn_sat['top_pct']<6, 5,
                         np.where(mn_sat['top_pct']<11, 10,
                                  np.where(mn_sat['top_pct']<21, 20,
                                           np.where(mn_sat['top_pct']<51, 50, 100))))
mn_sat = mn_sat.drop(['top_pct'], axis=1)
df = pd.merge(df, mn_sat, on='mn_sat', how='left')

### 2.3.3 Normalize Ratings and Preferences

#### 2.3.3.1 Normalize Ratings of Subject

In [31]:
idx_norm = [13,14,15,16,17,18,20,21,22,23,28,29,30,31,32,33]
for i in idx_norm:
    columns = list(df_columns[df_columns['cat no.']==i]['column'])
    ttl = df[columns].sum(axis=1)
    df2 = df[columns].div(other=ttl, axis=0)
    df2 = df2 * 100
    df.update(df2)


### 2.3.4 Success Rates

### 2.3.4.1. Success Rates for Subject

In [32]:
df['yes_rate'] = df.groupby('iid')['b_dec_o'].transform(lambda x:x.mean())
df['match_rate'] = df.groupby('iid')['b_match'].transform(lambda x:x.mean())

### Save data 
Save training, validation and test sets in an interim directory for exploratory data analysis

In [33]:
def write(df, directory, filename):
    if isinstance(df, pd.DataFrame):
        if isinstance(filename, str):
            if not os.path.isdir(directory):
                os.mkdir(directory)
            df.to_csv(os.path.join(directory, filename),
                      index=False, index_label=False)
            return(True)
        else:
            return(False)
    else:
        return(False)

In [34]:
write(df, "../data/interim", 'interim.csv')
write(df_labels, '../data/external', 'labels.csv')

True